In [1]:
import odbc
import win32com.client
import pandas as pd
import pyodbc

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)


In [2]:
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)

In [3]:
df

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None
...,...,...,...,...,...,...,...,...
34654,34695,Z:\Varios\,INTERNOS ACTUALIZADOS 2019.xlsx,2020-01-23 13:18:48.013,NaT,None,None,None
34655,34696,Z:\Varios\,stock en negativos 27 de septiembre.xlsx,2020-01-23 13:18:48.083,NaT,None,None,None
34656,34697,Z:\Ventas\,Libro1.xlsx,2020-01-23 13:18:48.230,NaT,None,None,None
34657,34698,Z:\Ventas\,presentacion 3-6 v2.xlsm,2020-01-23 13:18:48.250,NaT,None,None,None


In [4]:
def insert_file_connection(EXCEL_ID,NAME,STR_CONN,SQL):
    
    aux_cnn = connection()
    
    cursor = aux_cnn.cursor()
    
    params = EXCEL_ID,NAME,STR_CONN,SQL
    
    #query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%params
    query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%params
    #sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
    
    try:
        #cursor.execute(sql, params)
        cursor.execute(query)
        val = cursor.fetchval()
        cursor.commit()
        
        
        
    except Exception as e:
        print('*'*100)
        print(e)
        
    
    aux_cnn.close()
    return int(val)
    
    
    

In [41]:
def check_excel(file,excel_id,index):
    global output
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.EnableEvents = False
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)
            
        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)
                    try:
                        for o in x.Ranges(1).QueryTable.Parameters:


                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          o.SourceRange.Address,
                                          o.SourceRange.Parent.Name,
                                          o.SourceRange.Value,
                                          excel_id,
                                          index])
                        flag = 1
                    except:
                        flag = 0
                    if flag == 0:

                        try:
                            if len(i.ListObject.QueryTable.Parameters) != 0:

                                for p in i.ListObject.QueryTable.Parameters:

                                    datos.append([x,
                                                  x.ODBCConnection.CommandText,
                                                  x.ODBCConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value,
                                                  excel_id,
                                                  index])
                            else:

                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name, #EDIT None
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])

                        except Exception as e:
                            try:
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])
                            except:
                                try:
                                    if len(i.ListObject.QueryTable.Parameters) != 0:

                                        for p in i.ListObject.QueryTable.Parameters:
                                            datos.append([x,
                                                          x.OLEDBConnection.CommandText,
                                                          x.OLEDBConnection.Connection,
                                                          i.Parent.Name,
                                                          celda,
                                                          coordenada,
                                                          p.SourceRange.Address,
                                                          p.SourceRange.Parent.Name,
                                                          p.SourceRange.Value,
                                                          excel_id,
                                                          index])
                                    else:
                                        datos.append([x,
                                                      x.ODBCConnection.CommandText,
                                                      x.ODBCConnection.Connection,
                                                      i.Parent.Name, #EDIT None
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])

                                except:
                                    try:
                                        datos.append([x,
                                                      x.OLEDBConnection.CommandText,
                                                      x.OLEDBConnection.Connection,
                                                      i.Parent.Name,
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])
                                    except:                            
                                        datos.append([x,
                                                  None,
                                                  None,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  None,
                                                  None,
                                                  None,
                                                  excel_id,
                                                  index])

            

        except Exception as c:
            return c , []


        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None'])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
    
        if type(datos[0]) == str:
            
            output +=[datos]
        else:
            output += datos

        
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0
    except:
        return 'No se pudo abrir .xlsx',[]

In [6]:
import datetime
#datetime.datetime.today()

In [11]:
for i in range(201):
    if int(i/100) == i/100:
        print(i)

0
100
200


In [47]:
output = []
errores = []
start = datetime.datetime.today()
for index,row in df.iterrows():
    if int(index/1000) == index/1000:
        print(index)
#        if index > 0 :
#            raise
    archivo = row['path']+row['filename']
    #matar proceso excel
    result = check_excel(archivo,row['excel_id'],index)
    #result = check_excel('C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx')
    if result != 0:
        
        errores.append([row['excel_id'],archivo,result,index])
            
    else:
        pass
    #raise 
    #if index == 5:
        
end = datetime.datetime.today()
print(end-start)
raise
#     respuesta = result[0]
#     result[1]

0
1000
2000


KeyboardInterrupt: 

In [54]:
index

2168

In [68]:
len(errores) #2

51

In [27]:
df.loc[2]

excel_id                                                               42
path                     Z:\....Candidatos a borrar\Archivos Viejos\TODO\
filename                                   IMPRESION ROTULOS DEPOSITO.xls
date_of_insertion                              2020-01-23 12:10:12.707000
start_processing_date                                                 NaT
end_processing_date                                                  None
process_id                                                           None
error_msg                                                            None
Name: 2, dtype: object

In [446]:
print(df.loc[2]['path']+df.loc[2]['filename'])

Z:\....Candidatos a borrar\Archivos Viejos\TODO\IMPRESION ROTULOS DEPOSITO.xls


In [30]:
result = check_excel(df.loc[2]['path']+df.loc[2]['filename'],'40',4)

In [49]:
len(output)

9452

In [39]:
df.loc[2]

excel_id                                                               42
path                     Z:\....Candidatos a borrar\Archivos Viejos\TODO\
filename                                   IMPRESION ROTULOS DEPOSITO.xls
date_of_insertion                              2020-01-23 12:10:12.707000
start_processing_date                                                 NaT
end_processing_date                                                  None
process_id                                                           None
error_msg                                                            None
Name: 2, dtype: object

In [132]:
dfParametros.to_excel("relevamiento.xlsx")

In [50]:
dfParametros = pd.DataFrame(output)
dfParametros = dfParametros.rename({0: 'name_excel_connection',1: 'sql_excel_connection',2:'str_conn_excel_connection',3:'sheet_excel_connection_location',4:"Inicio_tabla_formula",5:'cell_excel_connection_location',6:'cell_excel_connection_location_parameter',7:"sheet_excel_connection_location_parameter",
                                    8:'content_excel_connection_location_parameter',9:'excel_id',10:'index'}, axis=1)

#dfParametros = dfParametros[dfParametros['Hoja']!= 'None']

In [ ]:
df[df['excel_id']==70]

In [67]:
print(df.loc[30]['path']+df.loc[30]['filename'])

Z:\Consultas\Articulos\Copia de stocks y demanda próximos 5 días.xlsx


In [64]:
dfParametros[dfParametros['excel_id']=='70'] # <------------------------

,name_excel_connection,sql_excel_connection,str_conn_excel_connection,sheet_excel_connection_location,Inicio_tabla_formula,cell_excel_connection_location,cell_excel_connection_location_parameter,sheet_connection_location_parameter,content_excel_connection_location_parameter,excel_id,index
55,Consulta desde ESTIMACIONES,"select a.codigo, ar.descrip, a.pt, a.pro, a.pr...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Hoja1,codigo,$A$1,None,None,None,70,30


In [56]:
dfParametros # <------------------------

,name_excel_connection,sql_excel_connection,str_conn_excel_connection,sheet_excel_connection_location,Inicio_tabla_formula,cell_excel_connection_location,cell_excel_connection_location_parameter,sheet_connection_location_parameter,content_excel_connection_location_parameter,excel_id,index
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Lista Tipos y Colores,codigo,$A$1,None,None,None,40,0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Lista Tipos y Colores,codigo,$D$1,None,None,None,40,0
2,None,None,None,None,None,None,None,None,None,None,None
3,Consulta desde ESTIMACIONES,"SELECT f.articulo, f.insumo, f.color, i.descri...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,BOLSAS,articulo,$A$1,None,None,None,42,2
4,Consulta desde ESTIMACIONES1,"SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PLANCHAR,ARTICULO,$A$4,None,None,None,42,2
...,...,...,...,...,...,...,...,...,...,...,...
9447,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$17,Posiciones diferencias x 20,None,2208,2168
9448,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$18,Posiciones diferencias x 20,None,2208,2168
9449,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$19,Posiciones diferencias x 20,None,2208,2168
9450,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$20,Posiciones diferencias x 20,None,2208,2168


In [53]:
len(errores)+len(output)

9503

In [412]:
dfParametros[dfParametros.isnull().any(1)]

,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,source_range_address,source_range_parent_name,source_range_value,excel_id,index


In [83]:
len(dfParametros), len(dfParametros.dropna())

(9452, 9452)

In [91]:
dfParametros = dfParametros[dfParametros['excel_id'] != 'None']

In [94]:
dfParametros['excel_id'] = dfParametros['excel_id'].astype(int)

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg
0,40,Z:\....Candidatos a borrar\Archivos Viejos\,LINEA 1118 - ORIGINAL (NO TOCAR).xlsm,2020-01-23 12:10:12.430,2020-01-24 16:13:16.420,None,3A8037B6-12FA-47D6-82D5-5C7B1F2F035D,None
1,41,Z:\....Candidatos a borrar\Archivos Viejos\,Observaciones.xlsm,2020-01-23 12:10:12.440,NaT,None,None,None
2,42,Z:\....Candidatos a borrar\Archivos Viejos\TODO\,IMPRESION ROTULOS DEPOSITO.xls,2020-01-23 12:10:12.707,NaT,None,None,None
3,43,Z:\....Candidatos a borrar\Carpetas compartidas\,primeros pedidos por zona 3.xlsx,2020-01-23 12:10:12.833,NaT,None,None,None
4,44,Z:\....Candidatos a borrar\consultas\movimientos\,Faltannu por fecha.xls,2020-01-23 12:10:12.963,NaT,None,None,None


In [415]:
resultado_df = pd.merge(df,dfParametros,on='excel_id',how='right')

In [416]:
len(dfParametros),len(resultado_df)

(6935, 6935)

In [418]:
resultado_df.to_excel('output_re1.xlsx')

In [393]:
resultado_df.tail()

,excel_id,path,filename,date_of_insertion,start_processing_date,end_processing_date,process_id,error_msg,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,10
16732,1410.0,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Conexión,"SELECT vida.articulo, vida.cp, vida.estado_cod...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,X cp,articulo,$A$5,$B$2,X cp,7355.0
16733,00693313008,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,7376.0
16734,29014502002,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,7377.0
16735,91203454007,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde ESTIMACIONES1,"select codigo, cantidad, frente, columna, nive...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Historial articulo,codigo,$A$4,$B$2,Historial articulo,7379.0
16736,91203454007,NaN,NaN,NaT,NaT,NaN,NaN,NaN,Consulta desde ESTIMACIONES2,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Historial articulo,codigo,$H$4,$B$2,Historial articulo,7379.0


In [373]:
dfParametros[dfParametros['excel_id'] == 3]

,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,excel_id,10


,Nombre,Texto de comando,Cadena de conexion,Hoja,Inicio_tabla_formula,Parametro,6,Contenido_Celda,excel_id
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,codigo,None,None,None,40,0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,codigo,None,None,None,40,0
2,None,None,None,None,None,None,None,None,41
3,Conexión,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TALLERES,codigo,None,None,None,42
4,Conexión1,"SELECT tipos.codigo, tipos.nombre, tipos.categ...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,TIPOS,codigo,None,None,None,42
...,...,...,...,...,...,...,...,...,...
104,None,None,None,None,None,None,None,None,79
105,Conexión,"SELECT caj_pedidos_por_dia.fecha, Sum(caj_pedi...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Pedidos por Día,2009-03-20 00:00:00+00:00,None,None,None,80
106,Conexión1,"SELECT caj_pedidos_por_dia.zona, caj_pedidos_p...",ODBC;DSN=ESTIMACIONES;Description=f;UID=estima...,Detalle del Día,0011,None,None,None,80
107,Conexión2,"SELECT caj_pedidos_por_dia.zona, caj_pedidos_p...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Detalle entre Fechas,2008-06-05 00:00:00+00:00,None,None,None,80


In [44]:
dfParametros['Texto de comando'][3]

'select * from estimaciones..movstock with (nolock) where fecha_pura = ? and movimiento = ?'

In [316]:
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
file = df.loc[40]['path']+df.loc[40]['filename']
wb = xl.Workbooks.Open(file, None, True)

datos = []

for x in wb.Connections:
        
    

    for i in x.Ranges:
        try:
            #PRIMER DATO QUE TRAE LA PRIMER CELDA
            celda = i.Cells(1,1)
            #LA COORDENADA DE LA PRIMER CELDA
            coordenada = i.Cells(1,1).Address
        except Exception as p:
            print(p)
        try:
            for o in x.Ranges(1).QueryTable.Parameters:
                
                
                datos.append([x,
                              x.ODBCConnection.CommandText,
                              x.ODBCConnection.Connection,
                              i.Parent.Name,
                              celda,
                              coordenada,
                              o.SourceRange.Address,
                              o.SourceRange.Parent.Name,
                              o.SourceRange.Value])
            flag = 1
        except:
            flag = 0
        if flag == 0:
            
            try:
                if len(i.ListObject.QueryTable.Parameters) != 0:

                    for p in i.ListObject.QueryTable.Parameters:

                        datos.append([x,
                                      x.ODBCConnection.CommandText,
                                      x.ODBCConnection.Connection,
                                      i.Parent.Name,
                                      celda,
                                      coordenada,
                                      p.SourceRange.Address,
                                      p.SourceRange.Parent.Name,
                                      p.SourceRange.Value])
                else:

                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  None,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])

            except Exception as e:
                try:
                    datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  i.Parent.Name,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])
                except:
                    try:
                        if len(i.ListObject.QueryTable.Parameters) != 0:

                            for p in i.ListObject.QueryTable.Parameters:
                                datos.append([x,
                                              x.OLEDBConnection.CommandText,
                                              x.OLEDBConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              p.SourceRange.Address,
                                              p.SourceRange.Parent.Name,
                                              p.SourceRange.Value])
                        else:
                            datos.append([x,
                                  x.ODBCConnection.CommandText,
                                  x.ODBCConnection.Connection,
                                  None,
                                  celda,
                                  coordenada,
                                  None,
                                  None,
                                  None])

                    except:
                        try:
                            datos.append([x,
                                          x.OLEDBConnection.CommandText,
                                          x.OLEDBConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          None,
                                          None,
                                          None])
                        except:                            
                            datos.append([x,
                                      None,
                                      None,
                                      i.Parent.Name,
                                      celda,
                                      coordenada,
                                      None,
                                      None,
                                      None])


        
        
    
    
    
    

'Detalle De Zona'

In [226]:
print(df.loc[40]['path']+df.loc[40]['filename'])

Z:\Consultas\CajasPicking\Pedidos.xls


'2009-03-20 00:00:00+00:00'

In [119]:
for i in x.Ranges:
    print(i)
    print(100*"*")

(('Suma de pedidos', 'Rótulos de columna', None, None, None), ('Rótulos de fila', 'EMPR', 'LIDR', 'LIDRUY', 'Total general'), (1902.0, None, 20940.0, None, 20940.0), (1903.0, 11029.0, 21347.0, None, 32376.0), (1904.0, 11920.0, 21714.0, None, 33634.0), (1905.0, None, 22398.0, None, 22398.0), (1906.0, 11829.0, 22420.0, None, 34249.0), (1907.0, 11801.0, 22232.0, None, 34033.0), (1908.0, None, 22466.0, None, 22466.0), (1909.0, 11411.0, 22507.0, None, 33918.0), (1910.0, 11212.0, 22898.0, None, 34110.0), (1911.0, None, 23415.0, None, 23415.0), (1912.0, 11097.0, 23478.0, None, 34575.0), (1913.0, 11837.0, 24717.0, None, 36554.0), (1914.0, 11404.0, 25071.0, None, 36475.0), (1915.0, None, 25208.0, None, 25208.0), (1916.0, 10734.0, 24885.0, None, 35619.0), (1917.0, 9178.0, 24221.0, 229.0, 33628.0), (1918.0, None, 22687.0, 375.0, 23062.0), (2001.0, 8746.0, 22537.0, 476.0, 31759.0), (2002.0, None, 23718.0, 658.0, 24376.0), (2003.0, 8903.0, 24160.0, 700.0, 33763.0), (2004.0, 9567.0, 24692.0, 820.0, 

In [115]:
print(i)

(('Suma de pedidos', 'Rótulos de columna', None, None, None), ('Rótulos de fila', 'EMPR', 'LIDR', 'LIDRUY', 'Total general'), (1902.0, None, 20940.0, None, 20940.0), (1903.0, 11029.0, 21347.0, None, 32376.0), (1904.0, 11920.0, 21714.0, None, 33634.0), (1905.0, None, 22398.0, None, 22398.0), (1906.0, 11829.0, 22420.0, None, 34249.0), (1907.0, 11801.0, 22232.0, None, 34033.0), (1908.0, None, 22466.0, None, 22466.0), (1909.0, 11411.0, 22507.0, None, 33918.0), (1910.0, 11212.0, 22898.0, None, 34110.0), (1911.0, None, 23415.0, None, 23415.0), (1912.0, 11097.0, 23478.0, None, 34575.0), (1913.0, 11837.0, 24717.0, None, 36554.0), (1914.0, 11404.0, 25071.0, None, 36475.0), (1915.0, None, 25208.0, None, 25208.0), (1916.0, 10734.0, 24885.0, None, 35619.0), (1917.0, 9178.0, 24221.0, 229.0, 33628.0), (1918.0, None, 22687.0, 375.0, 23062.0), (2001.0, 8746.0, 22537.0, 476.0, 31759.0), (2002.0, None, 23718.0, 658.0, 24376.0), (2003.0, 8903.0, 24160.0, 700.0, 33763.0), (2004.0, 9567.0, 24692.0, 820.0, 

In [317]:
if type(datos[0]) == list:
    datos = [list(map(str,i))for i in datos]

else:
    datos = list(map(str,datos))

In [327]:
result = pd.DataFrame(datos)

In [328]:
result = result.rename(columns={0:'nombre_conexion',1:'query',2:'str_connection',3:'parent_name',4:'first_value',5:'location_first_value',6:'location_parameter',7:'parent_name_parameter',8:'parameter_value'})

In [335]:
pd.DataFrame(datos)

,0,1,2,3,4,5,6,7,8,9,10
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$A$1,None,None,None,40,0.0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$D$1,None,None,None,40,0.0
2,None,None,None,None,None,None,None,None,41,1,NaN
3,Consulta desde ESTIMACIONES,"SELECT f.articulo, f.insumo, f.color, i.descri...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,articulo,$A$1,None,None,None,42,2.0
4,Consulta desde ESTIMACIONES1,"SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$4,None,None,None,42,2.0
5,Consulta desde estimaciones2,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,codigo,$H$3,None,None,None,42,2.0
6,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,39153736009,42,2.0
7,Consulta desde estimaciones4,SELECT DISTINCT ARTICULO FROM VIDA WHERE ESTAD...,ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$5,None,None,None,42,2.0
8,None,None,None,None,None,None,None,None,43,3,NaN
9,Conexión,"SELECT faltannu.cp, faltannu.pedido, faltannu....",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Faltante entre Fechas,cp,$A$8,$B$4,Faltante entre Fechas,2015-08-24 00:00:00+00:00,44,4.0


In [330]:
result

,nombre_conexion,query,str_connection,parent_name,first_value,location_first_value,location_parameter,parent_name_parameter,parameter_value,9,10
0,Consulta desde ESTIMACIONES,"SELECT tipos.codigo, tipos.nombre\r\nFROM ESTI...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$A$1,None,None,None,40,0.0
1,Consulta desde ESTIMACIONES1,"SELECT colores.codigo, colores.nombre\r\nFROM ...",ODBC;DSN=ESTIMACIONES;Description=estimaciones...,None,codigo,$D$1,None,None,None,40,0.0
2,None,None,None,None,None,None,None,None,41,1,NaN
3,Consulta desde ESTIMACIONES,"SELECT f.articulo, f.insumo, f.color, i.descri...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,articulo,$A$1,None,None,None,42,2.0
4,Consulta desde ESTIMACIONES1,"SELECT ARTICULO, DESCRIP , CANTIDAD FROM FORMU...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$4,None,None,None,42,2.0
5,Consulta desde estimaciones2,"SELECT taller.codigo, taller.nombre, taller.do...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,codigo,$H$3,None,None,None,42,2.0
6,Consulta desde estimaciones3,"SELECT procesos_calidad.articulo, procesos_cal...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,PROCESOS,articulo,$A$6,$D$3,PROCESOS,39153736009,42,2.0
7,Consulta desde estimaciones4,SELECT DISTINCT ARTICULO FROM VIDA WHERE ESTAD...,ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,None,ARTICULO,$A$5,None,None,None,42,2.0
8,None,None,None,None,None,None,None,None,43,3,NaN
9,Conexión,"SELECT faltannu.cp, faltannu.pedido, faltannu....",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Faltante entre Fechas,cp,$A$8,$B$4,Faltante entre Fechas,2015-08-24 00:00:00+00:00,44,4.0


In [71]:
############################################
## ESTO ES TEST PARA ENCONTRAR EL ERROR CATASTROFICO INDEX > 2000 y < 2170.
def test_excel(file,excel_id,index):
    global test_output
    #xl = win32com.client.gencache.EnsureDispatch('Excel.Application')
    #file = r'C:\Users\lrojas\Desktop\Copia de archivo de ejemplo.xlsx'
#    file = f'r{path}'
    try:
        xl = win32com.client.Dispatch('Excel.Application')
        xl.DisplayAlerts = False
        xl.EnableEvents = False
        try:
            #wb = xl.Workbooks.Open(file, None,True)
            #wb = xl.Workbooks.Open(Filename=file,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,Editable=False,ReadOnly=True)
            #wb = xl.Workbooks.Open(Filename=file,IgnoreReadOnlyRecommended=True,CorruptLoad=True,ReadOnly=True)
            wb = xl.Workbooks.Open(file, None, True)

        except Exception as e:
            #print(e)
            #xl.Application.Quit()
            return e , []

        datos = []
        try:
            for x in wb.Connections:
                for i in x.Ranges:
                    try:
                        #PRIMER DATO QUE TRAE LA PRIMER CELDA
                        celda = i.Cells(1,1)
                        #LA COORDENADA DE LA PRIMER CELDA
                        coordenada = i.Cells(1,1).Address
                    except Exception as p:
                        print(p)
                    try:
                        for o in x.Ranges(1).QueryTable.Parameters:


                            datos.append([x,
                                          x.ODBCConnection.CommandText,
                                          x.ODBCConnection.Connection,
                                          i.Parent.Name,
                                          celda,
                                          coordenada,
                                          o.SourceRange.Address,
                                          o.SourceRange.Parent.Name,
                                          o.SourceRange.Value,
                                          excel_id,
                                          index])
                        flag = 1
                    except:
                        flag = 0
                    if flag == 0:

                        try:
                            if len(i.ListObject.QueryTable.Parameters) != 0:

                                for p in i.ListObject.QueryTable.Parameters:

                                    datos.append([x,
                                                  x.ODBCConnection.CommandText,
                                                  x.ODBCConnection.Connection,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  p.SourceRange.Address,
                                                  p.SourceRange.Parent.Name,
                                                  p.SourceRange.Value,
                                                  excel_id,
                                                  index])
                            else:

                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name, #EDIT None
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])

                        except Exception as e:
                            try:
                                datos.append([x,
                                              x.ODBCConnection.CommandText,
                                              x.ODBCConnection.Connection,
                                              i.Parent.Name,
                                              celda,
                                              coordenada,
                                              None,
                                              None,
                                              None,
                                              excel_id,
                                              index])
                            except:
                                try:
                                    if len(i.ListObject.QueryTable.Parameters) != 0:

                                        for p in i.ListObject.QueryTable.Parameters:
                                            datos.append([x,
                                                          x.OLEDBConnection.CommandText,
                                                          x.OLEDBConnection.Connection,
                                                          i.Parent.Name,
                                                          celda,
                                                          coordenada,
                                                          p.SourceRange.Address,
                                                          p.SourceRange.Parent.Name,
                                                          p.SourceRange.Value,
                                                          excel_id,
                                                          index])
                                    else:
                                        datos.append([x,
                                                      x.ODBCConnection.CommandText,
                                                      x.ODBCConnection.Connection,
                                                      i.Parent.Name, #EDIT None
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])

                                except:
                                    try:
                                        datos.append([x,
                                                      x.OLEDBConnection.CommandText,
                                                      x.OLEDBConnection.Connection,
                                                      i.Parent.Name,
                                                      celda,
                                                      coordenada,
                                                      None,
                                                      None,
                                                      None,
                                                      excel_id,
                                                      index])
                                    except:                            
                                        datos.append([x,
                                                  None,
                                                  None,
                                                  i.Parent.Name,
                                                  celda,
                                                  coordenada,
                                                  None,
                                                  None,
                                                  None,
                                                  excel_id,
                                                  index])

            

        except Exception as c:
            return c , []


        #df = pd.DataFrame(datos)
        #df = df.applymap(str)
        #datos = map(str,datos)
        if datos == []:
            datos.append(['None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None',
                          'None'])
        elif type(datos[0]) == list:
            datos = [list(map(str,i))for i in datos]

        else:
            datos = list(map(str,datos))
    #    xl.DisplayAlerts = False
    
        if type(datos[0]) == str:
            
            test_output +=[datos]
        else:
            test_output += datos

        
        try:
            #wb.Close(0)
            #wb.Close(False)
            #wb.Close(SaveChanges=0)
            xl.ActiveWorkbook.Saved = True
            xl.ActiveWorkbook.Close
            #wb.Close(SaveChanges=0,Filename=file)
        except:
            wb.Close(SaveChanges = False)
            #wb.Close(SaveChanges = False,Filename=file)
        xl.Application.Quit()

        #return df
        return 0
    except:
        return 'No se pudo abrir .xlsx',[]

In [76]:
test_output = []
test_errores = []
start = datetime.datetime.today()

for ix,row in df.loc[2000:2170].iterrows():
    
    archivo = row['path']+row['filename']
    #matar proceso excel
    result = test_excel(archivo,row['excel_id'],ix)
    #result = check_excel('C:\\Users\\jhernandez\\Desktop\\Copia de archivo de ejemplo.xlsx')
    if result != 0:
        
        test_errores.append([row['excel_id'],archivo,result,ix])
            
    else:
        pass
    #raise 
    #if index == 5:
        
end = datetime.datetime.today()
print(end-start)
raise
#     respuesta = result[0]
#     result[1]

0:21:56.764059


RuntimeError: No active exception to reraise

In [82]:
len(index)

TypeError: object of type 'int' has no len()

In [117]:
index

2168

In [77]:
len(test_errores)

19

In [78]:
len(test_output)

517

In [ ]:
#Para el control de procesos ¡¡¡REVISAR!!!

import os
r = os.popen('tasklist /v').read().strip().split('\n')
name  = "saplogon.exe"
for i in range(len(r)):
    if name in r[i]:
        os.system("taskkill /im %s" %(name))
        
#fuente: https://stackoverflow.com/questions/48866010/python-kill-a-program-but-check-first

In [81]:
dataoutput= pd.DataFrame(test_output)
dataoutput

,0,1,2,3,4,5,6,7,8,9,10
0,Consulta desde ESTIMACIONES,SELECT * \r\nFROM lavjean_apertura_envios_co...,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,LavaderosJean,deposito,$A$4,None,None,None,2040,2000
1,Consulta desde ESTIMACIONES2,SELECT arreglos_apertura_envios_detallado.arti...,ODBC;DSN=ESTIMACIONES;UID=consultas;APP=2007 M...,Arreglos,articulo,$A$4,None,None,None,2040,2000
2,Consulta desde ESTIMACIONES3,SELECT * \r\nFROM ESTIMACIONES.dbo.embolsado_a...,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Embolsado,proveedor,$A$6,None,None,None,2040,2000
3,Consulta desde ESTIMACIONES4,SELECT * \r\nFROM ESTIMACIONES.dbo.atracadores...,ODBC;DSN=ESTIMACIONES;Description=estimaciones...,Atracadores e IPR,proveedor,$A$4,None,None,None,2040,2000
4,Consulta desde estimaciones5,SELECT * \r\nFROM botonera_apertura_envios_con...,ODBC;DSN=estimaciones;Description=estimaciones...,Botonera,deposito,$A$6,None,None,None,2040,2000
...,...,...,...,...,...,...,...,...,...,...,...
512,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$17,Posiciones diferencias x 20,None,2210,2170
513,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$18,Posiciones diferencias x 20,None,2210,2170
514,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$19,Posiciones diferencias x 20,None,2210,2170
515,Consulta desde ESTIMACIONES4,"select codigo,cantidad,frente,columna,nivel,ca...",ODBC;DSN=ESTIMACIONES;Description=ESTIMACIONES...,Posiciones diferencias x 20,codigo,$D$2,$B$20,Posiciones diferencias x 20,None,2210,2170


In [105]:
#2000 a 20170
notIn = []
for i in range(2000,2171):
    if str(i) not in dataoutput[10].unique().tolist():
        notIn.append(i)

In [130]:
print(df.loc[2008]['path']+df.loc[2008]['filename'])

Z:\Consultas\Diego\premios empresaris.xlsx


In [ ]:
# Lista de posibles erroes en excel al abrir (error)
#Z:\Consultas\Estimaciones\VIEJO\Posibles Sobrantes para Gondola.xlsx
#Z:\Consultas\Estimaciones\Proyección de pedidos-unidades sobre zonas ingresadas 2 - no funciona 20160215.xlsx
#Z:\Consultas\Mano de Obra\Actualizador de Mano de Obra.xlsx

In [115]:
for m in notIn:
    print(df.loc[m]['path']+df.loc[m]['filename'])

Z:\Consultas\Despacho\VARIOS\Copia de POSICIONES FRENTE-COLUMNA-NIVEL_RECUPERADO 1.xls
Z:\Consultas\Despacho\VARIOS\POSICIONES FRENTE-COLUMNA-NIVEL.xls
Z:\Consultas\Despacho\VARIOS\POSICIONES FRENTE-COLUMNA-NIVEL_RECUPERADO.xls
Z:\Consultas\Diego\Copia de HOJA DE RUTA DIEGO.xlsx
Z:\Consultas\Diego\llamados andreani.xlsx
Z:\Consultas\Diego\premios empresaris.xlsx
Z:\Consultas\Diego\CAT0314\BASE CAT 030414.xlsx
Z:\Consultas\Diego\GUIAS REG\R 12.xlsx
Z:\Consultas\Diego\GUIAS REG\R 14.xlsx
Z:\Consultas\Diego\GUIAS REG\R 2.xlsx
Z:\Consultas\Diego\GUIAS REG\R 6.xlsx
Z:\Consultas\Diego\GUIAS REG\R 62.xlsx
Z:\Consultas\Diego\GUIAS REG\R 7.xlsx
Z:\Consultas\Diego\GUIAS REG\R 82.xlsx
Z:\Consultas\Diego\GUIAS REG\R 96.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 12.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 14.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 2.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 6.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 62.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 7.xlsx
Z:\Co

In [118]:
#2000 a 20170
In = []
for i in range(2000,2171):
    if str(i) in dataoutput[10].unique().tolist():
        In.append(i)
        
for m in In:
    print(df.loc[m]['path']+df.loc[m]['filename'])

Z:\Consultas\Despacho\PRO\Stocks Varios x Envios (Botonera-LavJean-Atracad-Embolsado) con cant botones.xlsm
Z:\Consultas\Diego\datos zonas.xlsx
Z:\Consultas\Diego\INFO ZONAS NAN 1.xlsx
Z:\Consultas\Diego\BACK UP DIEGO\Calendario  2014.xlsx
Z:\Consultas\Estimaciones\Demanda Modelo Semanal.xls
Z:\Consultas\Estimaciones\Histórico Feria Americana - VIEJO.xls
Z:\Consultas\Estimaciones\Abastecimiento Gondola.xlsx
Z:\Consultas\Estimaciones\articulos faltantes en linea D.xlsx
Z:\Consultas\Estimaciones\articulos faltantes en linea.xlsx
Z:\Consultas\Estimaciones\Balanceo de Linea.xlsx
Z:\Consultas\Estimaciones\Campañas vigentes en Proyeccion.xlsx
Z:\Consultas\Estimaciones\Candidatos a unificar cajas.xlsx
Z:\Consultas\Estimaciones\Copia de Listado de Premios.xlsx
Z:\Consultas\Estimaciones\Copia de Proyección de pedidos-unidades sobre zonas ingresadas 3.xlsx
Z:\Consultas\Estimaciones\Devoluciones vs Reemplazos.xlsx
Z:\Consultas\Estimaciones\Estimacion Uso Ordenes de Trabajo.xlsx
Z:\Consultas\Estim

In [139]:
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
xl.EnableEvents = False
for m in In:
    
    wb = xl.Workbooks.Open(df.loc[m]['path']+df.loc[m]['filename'], None, True)
    wb.Close(SaveChanges = False)

xl.Application.Quit()

In [138]:
In

[2000,
 2005,
 2006,
 2009,
 2029,
 2030,
 2031,
 2032,
 2033,
 2034,
 2035,
 2036,
 2037,
 2038,
 2040,
 2041,
 2042,
 2043,
 2044,
 2045,
 2046,
 2047,
 2048,
 2049,
 2050,
 2052,
 2053,
 2054,
 2055,
 2056,
 2057,
 2059,
 2060,
 2061,
 2063,
 2064,
 2065,
 2066,
 2067,
 2068,
 2069,
 2070,
 2073,
 2080,
 2083,
 2084,
 2087,
 2088,
 2090,
 2094,
 2095,
 2096,
 2097,
 2098,
 2100,
 2102,
 2103,
 2104,
 2105,
 2106,
 2110,
 2111,
 2112,
 2113,
 2114,
 2115,
 2116,
 2118,
 2119,
 2121,
 2123,
 2124,
 2125,
 2126,
 2127,
 2128,
 2129,
 2130,
 2131,
 2132,
 2134,
 2135,
 2136,
 2137,
 2138,
 2139,
 2140,
 2141,
 2142,
 2143,
 2144,
 2147,
 2148,
 2149,
 2150,
 2151,
 2152,
 2153,
 2154,
 2156,
 2157,
 2159,
 2160,
 2161,
 2162,
 2163,
 2164,
 2165,
 2166,
 2168,
 2170]

In [247]:
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
xl.EnableEvents = False

wb = xl.Workbooks.Open('Z:\Consultas\Diego\GUIAS REG\R 82.xlsx', None, True)
wb.Close(SaveChanges = False)

xl.Application.Quit()


In [141]:
for m in notIn:
    print(df.loc[m]['path']+df.loc[m]['filename'])

Z:\Consultas\Despacho\VARIOS\Copia de POSICIONES FRENTE-COLUMNA-NIVEL_RECUPERADO 1.xls
Z:\Consultas\Despacho\VARIOS\POSICIONES FRENTE-COLUMNA-NIVEL.xls
Z:\Consultas\Despacho\VARIOS\POSICIONES FRENTE-COLUMNA-NIVEL_RECUPERADO.xls
Z:\Consultas\Diego\Copia de HOJA DE RUTA DIEGO.xlsx
Z:\Consultas\Diego\llamados andreani.xlsx
Z:\Consultas\Diego\premios empresaris.xlsx
Z:\Consultas\Diego\CAT0314\BASE CAT 030414.xlsx
Z:\Consultas\Diego\GUIAS REG\R 12.xlsx
Z:\Consultas\Diego\GUIAS REG\R 14.xlsx
Z:\Consultas\Diego\GUIAS REG\R 2.xlsx
Z:\Consultas\Diego\GUIAS REG\R 6.xlsx
Z:\Consultas\Diego\GUIAS REG\R 62.xlsx
Z:\Consultas\Diego\GUIAS REG\R 7.xlsx
Z:\Consultas\Diego\GUIAS REG\R 82.xlsx
Z:\Consultas\Diego\GUIAS REG\R 96.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 12.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 14.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 2.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 6.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 62.xlsx
Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 7.xlsx
Z:\Co

In [ ]:
# excels con errores catastroficos
# Z:\Consultas\Estimaciones\Reporte Estimaciones.xlsm

#Error:

'''---------------------------------------------------------------------------
com_error                                 Traceback (most recent call last)
<ipython-input-176-00e89761f30e> in <module>
      3 xl.EnableEvents = False
      4 
----> 5 wb = xl.Workbooks.Open('Z:\Consultas\Estimaciones\Reporte Estimaciones.xlsm', None, True)
      6 wb.Close(SaveChanges = False)
      7 

C:\ProgramData\Anaconda3\lib\site-packages\win32com\client\dynamic.py in Open(self, Filename, UpdateLinks, ReadOnly, Format, Password, WriteResPassword, IgnoreReadOnlyRecommended, Origin, Delimiter, Editable, Notify, Converter, AddToMru, Local, CorruptLoad)

com_error: (-2147023170, 'Error en la llamada a procedimiento remoto.', None, None)'''

# error, no finaliza, al matar proceso salta ERROR CATASTROFICO
# Z:\Consultas\Insumos\ESTIMADOS REUTILIZACION DISCONTINUOS\prueba Pablo.xlsm

# Debug de error catastrofico

In [258]:
ESCOPETAZO = True

foo = ['Z:\Consultas\Estimaciones\Reporte Estimaciones.xlsm',
       'Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 7.xlsx',
       'Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 82.xlsx',
       'Z:\Consultas\Diego\GUIAS REG\VIEJOS\R 96.xlsx']
for f in foo:
    
    xl = win32com.client.Dispatch('Excel.Application')
    xl.DisplayAlerts = False
    xl.EnableEvents = False

    #ejecutar monitoreo.py
    try:
        wb = xl.Workbooks.Open(f, None, True)
    except Exception as e:
        print(e)
    try:
        wb.Close(SaveChanges = False)

        xl.Application.Quit()
    except:
        pass
%shutdown notebook con while True
    

(-2147023170, 'Error en la llamada a procedimiento remoto.', None, None)


In [242]:
xl = win32com.client.Dispatch('Excel.Application')
xl.DisplayAlerts = False
xl.EnableEvents = False

wb = xl.Workbooks.Open('Z:\Consultas\Insumos\ESTIMADOS REUTILIZACION DISCONTINUOS\prueba Pablo.xlsm', None, True)
wb.Close(SaveChanges = False)

xl.Application.Quit()

com_error: (-2147023170, 'Error en la llamada a procedimiento remoto.', None, None)

In [ ]:
* bandera_kill = 0
* ejecutamos relevamiento excel
* % run monitoreo.py
* terminé bandera_kill = 1
while bandera_kill != 1:
    
    ss
    0

* cuarentena opcion B. 



In [237]:
import psutil
psutil.process_iter(attrs=None, ad_value=None)
for proc in psutil.process_iter():
    try:
        # Get process name & pid from process object.
        processName = proc.name()
        processID = proc.pid
        print(processName , ' ::: ', processID)
    except (psutil.NoSuchProcess, psutil.AccessDenied, psutil.ZombieProcess):
        pass

System Idle Process  :::  0
System  :::  4
Registry  :::  84
chrome.exe  :::  304
smss.exe  :::  320
dwm.exe  :::  396
csrss.exe  :::  436
wininit.exe  :::  544
csrss.exe  :::  552
svchost.exe  :::  556
services.exe  :::  612
lsass.exe  :::  620
svchost.exe  :::  724
ctfmon.exe  :::  748
svchost.exe  :::  756
fontdrvhost.exe  :::  764
svchost.exe  :::  832
svchost.exe  :::  884
taskhostw.exe  :::  896
chrome.exe  :::  900
winlogon.exe  :::  932
fontdrvhost.exe  :::  988
svchost.exe  :::  1000
chrome.exe  :::  1004
YourPhone.exe  :::  1044
svchost.exe  :::  1056
svchost.exe  :::  1096
svchost.exe  :::  1124
svchost.exe  :::  1132
svchost.exe  :::  1140
svchost.exe  :::  1148
svchost.exe  :::  1156
svchost.exe  :::  1284
svchost.exe  :::  1300
svchost.exe  :::  1328
chrome.exe  :::  1356
svchost.exe  :::  1360
svchost.exe  :::  1376
svchost.exe  :::  1388
svchost.exe  :::  1420
svchost.exe  :::  1464
svchost.exe  :::  1472
chrome.exe  :::  1524
svchost.exe  :::  1600
svchost.exe  :::  16

In [ ]:
def si_excel():
    import win32ui
    # may need FindWindow("iTunes", None) or FindWindow(None, "iTunes")
    # or something similar
    if FindWindow("iTunes", "iTunes"):
        print "Found an iTunes window"
        return True

import odbc
import win32com.client
import pandas as pd
import pyodbc

def connection():
    server = 'tcp:192.168.1.6'
    database = 'planeamiento'
    username = 'sa'
    password = 'sa'

    return pyodbc.connect('DRIVER={SQL Server};SERVER='+server+';DATABASE='+database+';UID='+username+';PWD='+ password)
cnxn = connection()
query = 'select * from planeamiento..ctrl_excel where end_processing_date is null'

df = pd.read_sql(query,cnxn)
df
def insert_file_connection(EXCEL_ID,NAME,STR_CONN,SQL):
    
    aux_cnn = connection()
    
    cursor = aux_cnn.cursor()
    
    params = EXCEL_ID,NAME,STR_CONN,SQL
    
    #query = "EXEC planeamiento..ctrl_excel_InsertFileConnection @excel_id = %s, @name = '%s', @str_conn = '%s', @sql = '%s'"%params
    query = """EXEC planeamiento..ctrl_excel_InsertFileConnection  %s, '%s', '%s', '%s'"""%params
    #sql = "{CALL planeamiento..ctrl_excel_InsertFileConnection (?,?,?,?)}"
    
    try:
        #cursor.execute(sql, params)
        cursor.execute